In [1]:
import numpy as np
import pandas as pd
import sklearn
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pickle

# Spacy
import spacy
nlp=spacy.load('es_core_news_sm')

# Stemmer
from nltk.stem import SnowballStemmer
spanish_stemmer = SnowballStemmer('spanish')

In [2]:
respuestas = pd.read_csv('tbl_respuestas.csv', sep=',', encoding='utf_8')
respuestas.drop("Modalidad", axis=1, inplace=True)
respuestas

,IDRespuesta,Intencion,SubIntencion,Carrera,w5,Respuesta
0,1,generalidades,todas,IA,todas,A partir de la captación y procesamiento de da...
1,2,generalidades,todas,MM,todas,"Como Diseñador de Multimedios, serás un profes..."
2,3,generalidades,todas,DG,todas,El diseño gráfico es una disciplina indispensa...
3,4,generalidades,todas,Pub,todas,"En la actualidad y desde siempre, la publicida..."
4,5,generalidades,todas,RRPP,todas,Las Relaciones Públicas constituyen un conjunt...
...,...,...,...,...,...,...
81,82,tramites,todas,todas,donde,\nLa preinscripción puede hacerse a través de ...
82,83,tramites,preinscripcion,todas,donde,\nLa preinscripción puede hacerse a través de ...
83,84,tramites,cies,todas,donde,La preinscripción puede hacerse a través de nu...
84,85,tramites,requisitos,todas,donde,La preinscripción puede hacerse a través de nu...


In [3]:
#pedile al profe si puede pasar los enlaces de programacion logica.
aa = input("texto: ")

In [4]:
df_textos = pd.DataFrame(columns = ['oracion'])
df_textos.loc[0] = [aa] 
df_textos

,oracion
0,quiero estudiar algo relacionado a la computacion


In [5]:
def PreProcesar(Corpus, POS=False, Lema=True, Stem=True):
    
    # Generar una lista de documentos de spacy para tratar el POS Tagging y la Lematización
    docs=[]
    for oracion in Corpus:
        docs.append(nlp(oracion.lower())) #La lematización funciona mejor en minúsculas
    
    # Crear una lista de oraciones, donde cada elemento es una lista de palabras.
    # Cada palabra está definida por una tupla (Texto, POSTag, Lema)
    # Se omiten los tokens que son identificados como signos de puntuación
    oraciones=[]
    for doc in docs:
        oracion=[]
        for token in doc:
            if token.pos_ != 'PUNCT':
                oracion.append((token.text, token.pos_, token.lemma_))
        oraciones.append(oracion)
    
    fc = open('stopwords.txt', 'r', encoding='utf8')
    stopwords = fc.read().split('\n')
    fc.close()
    oraciones = [[palabra for palabra in oracion if palabra[2] not in stopwords] for oracion in oraciones]
    
    # Stemming
    if Stem==True:
        oraciones_aux=[]
        for oracion in oraciones:
            oracion_aux=[]
            for palabra in oracion:
                p_texto, p_pos, p_lema = palabra
                # Si Lema es True, se Stemmatiza el lema; si no, se Stemmatiza la palabra original
                if Lema==True:
                    oracion_aux.append((p_texto, p_pos, p_lema, spanish_stemmer.stem(p_lema)))
                else:
                    oracion_aux.append((p_texto, p_pos, p_lema, spanish_stemmer.stem(p_texto)))
            oraciones_aux.append(oracion_aux)
        
        oraciones = oraciones_aux
    
    # Finalmente: devolver nuevamente una lista de cadenas como la recibida, pero con el contenido
    # de cada cadena conformado según los parámetros:
    
    Corpus_Procesado = [] #Variable de salida
    
    for doc in oraciones:
        oracion = ''
        for palabra in doc:
            if Stem == True:
                # Devolver cadena de Stemming
                oracion = oracion + palabra[3]
            else:
                if Lema == True:
                    # Devolver cadena de Lemas
                    oracion = oracion + palabra[2]
                else:
                    # Devolver cadena de palabras originales
                    oracion = oracion + palabra[0]
            
            if POS == True:
                #Concatenar POS a cada palabra
                oracion = oracion + '_' + palabra[1].lower()
            
            oracion = oracion + ' '
        
        Corpus_Procesado.append(oracion)
        
    return Corpus_Procesado

def Corregir_Documentos(df_textos, columnas, POS=False, Lema=True, Stem=True):

    for col in columnas:
        df_textos[col] = PreProcesar(list(df_textos[col]), POS, Lema, Stem)
    
    # Sanear el DataFrame eliminando los duplicados y reindexándolo
    df_textos = df_textos.drop_duplicates().reset_index(drop=True)
    
    return df_textos

def Generar_Matriz_BOW(df_textos, columna, binario=False, ngram=(1,2)):
    
    # Vectorizar, usando CountVectorizer de sklearn.feature_extraction.text
    from sklearn.feature_extraction.text import CountVectorizer
    vectorizador = CountVectorizer(binary=binario, ngram_range=ngram)
    X = vectorizador.fit_transform(df_textos[columna])
    
    # Generar el DataFrame a devolver
    df_X = pd.DataFrame(X.toarray(), columns=vectorizador.get_feature_names())
    df = df_textos.join(df_X)
    
    return vectorizador, df

def Generar_Matriz_Tfidf(df_textos, columna, ngram=(1,2)):
    
    # Vectorizar... Directamente usar aquí el TfidfVectorizer de sklearn en vez del CountVectorizer
    # (Lleva los mismos parámetros y directamente nos devuelve la matriz con los vectores Tf*Idf)
    from sklearn.feature_extraction.text import TfidfVectorizer
    vectorizador = TfidfVectorizer(ngram_range=ngram)
    X = vectorizador.fit_transform(df_textos[columna])
    
    # Generar el DataFrame a devolver
    df_X = pd.DataFrame(X.toarray(), columns=vectorizador.get_feature_names())
    df = df_textos.join(df_X)
    
    return vectorizador, df